conda install openpyxl

In [1]:
import pandas as pd  
from datetime import datetime, timedelta
import re
import os

In [61]:
#กำหนด input file / ชื่อ output file
input='RawDataCSV\Biore UV Aqua Rich Watery Gel_20240912_160935.csv'
output='Biore UV Aqua Rich Watery Gel_20240912_160935.csv'

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\earns\AppData\Local\Temp\ipykernel_10608\301382224.py:2: SyntaxWarning: invalid escape sequence '\B'
  input='RawDataCSV\Biore UV Aqua Rich Watery Gel_20240912_160935.csv'


In [48]:
def convert_date(date_str, file_date):
    # แปลงวันที่จากชื่อไฟล์
    file_date = datetime.strptime(file_date, '%Y%m%d')
    
    # กรณีที่มีคำว่า 'ก่อน'
    if 'ก่อน' in date_str:
        value, unit = date_str.split()[:2]
        value = int(value)
        
        if unit in ['วินาทีก่อน', 'นาทีก่อน', 'ชั่วโมงก่อน']:
            return file_date.strftime('%Y%m%d')
        elif unit == 'วันก่อน':
            return (file_date - timedelta(days=value)).strftime('%Y-%m-%d')
        elif unit == 'สัปดาห์ก่อน':
            return (file_date - timedelta(weeks=value)).strftime('%Y-%m-%d')
    
    month_dict = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }
    
    parts = date_str.split()
    if len(parts) == 3 and parts[1] in month_dict:
        day, month, year = parts
        return f"{year}-{month_dict[month]}-{int(day):02d}"
    
    return date_str

def process_dataframe(file_path):
    # อ่านไฟล์ CSV
    df = pd.read_csv(file_path)
    
    # แยกวันที่จากชื่อไฟล์
    file_date = file_path.split('_')[-2]
    
    # แปลงคอลัมน์ date
    df['date'] = df['date'].apply(lambda x: convert_date(x, file_date))
    
    return df


In [51]:
df = process_dataframe(input)
print(df)

    no       name  rating        date  \
0    1      จ***.       5  2024-08-29   
1    2      P***i       5  2024-06-23   
2    3      N***V       5  2024-04-09   
3    4      g***t       5  2024-07-18   
4    5      9***1       5  2024-04-03   
5    6      N***V       5  2024-04-09   
6    7      9***1       5  2024-04-03   
7    8       Kaew       5  2024-06-11   
8    9      L***.       5  2024-05-08   
9   10      S***.       5  2024-07-03   
10  11      P***.       5  2024-05-10   
11  12      ว***.       5  2024-03-24   
12  13      D***.       5  2022-02-24   
13  14      P***a       5  2022-02-17   
14  15  ศิโรรัตน์       5  2024-03-04   
15  16      P***A       5  2024-09-05   
16  17      B***.       5  2024-08-29   
17  18  วิศรุต เ.       5  2024-08-29   
18  19      N***.       5  2024-08-11   
19  20      T***N       5  2023-11-20   
20  21   วาสนา ว.       5  2024-07-15   
21  22     Isa S.       5  2024-07-14   
22  23      ~***9       5  2024-03-07   
23  24      น***

In [53]:
df.dtypes

no              int64
name           object
rating          int64
date           object
review_text    object
dtype: object

In [54]:
# ฟังก์ชันลบข้อมูลที่ไม่ใช่ภาษาไทย
def clean_review(text):
    # ลบอีโมจิ, ภาษาอังกฤษ, ตัวเลข, สัญลักษณ์พิเศษ และช่องว่างเกินจำเป็น
    cleaned_text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)  # \u0E00-\u0E7F คือตัวอักษรภาษาไทยใน Unicode
    return cleaned_text.strip()

# ใช้ฟังก์ชัน clean_review กับคอลัมน์ review_text
df['review_text'] = df['review_text'].apply(lambda x: clean_review(str(x)))

In [55]:
# ลบการขึ้นบรรทัดใหม่ใน review_text และทำให้ข้อความอยู่ในบรรทัดเดียวกัน
df['review_text'] = df['review_text'].apply(lambda x: str(x).replace('\n', ' ').strip())

In [56]:
# ลบแถวที่ review_text ว่าง (ที่ไม่มีข้อมูลภาษาไทยหรือเป็นช่องว่าง)
df_cleaned = df[df['review_text'].astype(bool)].copy()

In [57]:
df_cleaned

,no,name,rating,date,review_text
0,1,จ***.,5,2024-08-29,ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...
1,2,P***i,5,2024-06-23,สะใจมากกันแดดขวดนี้ปริมาณ กดมาได้ในราคา บาท...
3,4,g***t,5,2024-07-18,สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก...
4,5,9***1,5,2024-04-03,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...
6,7,9***1,5,2024-04-03,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...
7,8,Kaew,5,2024-06-11,ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึ...
8,9,L***.,5,2024-05-08,ทำให้ผิวชุ่มชื้น สูตรที่อยู่ได้นาน ปกป้องผิวจา...
9,10,S***.,5,2024-07-03,ใช้ประจำป้องกันแดดได้ดร เนื้อสัมผัสดีมากๆ ราคา...
10,11,P***.,5,2024-05-10,ทำให้ผิวดูสุขภาพดีและเป็นธรรมชาติ ปกป้องผิวจาก...
11,12,ว***.,5,2024-03-24,เหมาะสำหรับใช้ในชีวิตประจำวัน ไม่ทำให้เกิดสิว ...


In [58]:
# รีเซ็ตเลขลำดับคอลัมน์ 'no' ใน DataFrame ที่ผ่านการตรวจสอบ
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['no'] = df_cleaned.index + 1

In [59]:
df_cleaned

,no,name,rating,date,review_text
0,1,จ***.,5,2024-08-29,ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...
1,2,P***i,5,2024-06-23,สะใจมากกันแดดขวดนี้ปริมาณ กดมาได้ในราคา บาท...
2,3,g***t,5,2024-07-18,สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก...
3,4,9***1,5,2024-04-03,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...
4,5,9***1,5,2024-04-03,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...
5,6,Kaew,5,2024-06-11,ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึ...
6,7,L***.,5,2024-05-08,ทำให้ผิวชุ่มชื้น สูตรที่อยู่ได้นาน ปกป้องผิวจา...
7,8,S***.,5,2024-07-03,ใช้ประจำป้องกันแดดได้ดร เนื้อสัมผัสดีมากๆ ราคา...
8,9,P***.,5,2024-05-10,ทำให้ผิวดูสุขภาพดีและเป็นธรรมชาติ ปกป้องผิวจาก...
9,10,ว***.,5,2024-03-24,เหมาะสำหรับใช้ในชีวิตประจำวัน ไม่ทำให้เกิดสิว ...


In [ ]:
if not os.path.exists('CleanDataCSV'):
    os.makedirs('CleanDataCSV')
if not os.path.exists('CleanDataExel'):
    os.makedirs('CleanDataExel')

In [62]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'CleanDataCSV/{output}.csv'
df_cleaned.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'CleanDataExel/{output}.xlsx'
df_cleaned.to_excel(excel_file, index=False)